In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
!pip install xlrd
!pip install autoviz
from autoviz.AutoViz_Class import AutoViz_Class
#Instantiate the AutoViz class
AV = AutoViz_Class()


# Reading the training set data

In [ ]:
data = pd.read_csv("/kaggle/input/mobile-price-classification/train.csv")
data.head()


In [ ]:
df_viz = AV.AutoViz("/kaggle/input/mobile-price-classification/train.csv",  depVar = 'price_range')

# Checking for null values
No Null values found

In [ ]:
data.isnull().any().sum()

In [ ]:
data.describe()

# Plotting Heatmap

In [ ]:
plt.figure(figsize=(10,7))
corr = data.corr()
sns.heatmap(corr, cmap="YlGnBu", linewidth = 0.7)

> *** Corelation exists, Price depends upon ram & somewhat with battery power***

# Plotting pair plots with most corelated parameters only

In [ ]:
sns.pairplot(data, vars = ['battery_power','px_height', 'px_width', 'ram','pc','fc'], hue = 'price_range', palette = 'coolwarm')

# Finding affect of mobile wt & int_memory with price range

In [ ]:
g = sns.FacetGrid(data, col="price_range", hue="wifi", palette = 'mako')
g.map(sns.scatterplot, "mobile_wt", "int_memory", alpha=.7)
plt.figure(figsize=(15,7))
g.add_legend()

# Finding affect of ram & int_memory with price range

In [ ]:
g = sns.FacetGrid(data, col="price_range", hue="wifi", palette = 'deep')
g.map(sns.scatterplot, "ram", "int_memory", alpha=.7)
plt.figure(figsize=(15,7))
g.add_legend()

# Finding affect of ram & battery power with price range

In [ ]:
g = sns.FacetGrid(data, col="price_range", hue="four_g", palette = 'rocket')
g.map(sns.scatterplot, "ram", "battery_power", alpha=.7)
plt.figure(figsize=(15,7))
g.add_legend()

In [ ]:
g = sns.FacetGrid(data, col="price_range", hue="n_cores")
g.map(sns.scatterplot, "ram", "px_height", alpha=.7)
plt.figure(figsize=(15,7))
g.add_legend()

# reading the test data & removing id column

In [ ]:
test = pd.read_csv("/kaggle/input/mobile-price-classification/test.csv")
test = test.drop(['id'], axis = 1)
test.head()

In [ ]:
test.info()

# Train Test & Split

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X = data.drop(['price_range'], axis = 1)
y = data.price_range

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 43)

# Converting data into standard scaler

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
scaler = StandardScaler()
scaler.fit(X_train, y_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

# Model RandomForest Classifier

In [ ]:
classifier=RandomForestClassifier(n_estimators=100)
model = classifier.fit(X_train, y_train)
y_pred = model.predict(X_test)

In [ ]:
df = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
df

In [ ]:
from sklearn import metrics
from sklearn.metrics import r2_score

# Accuracy & R2 Score

In [ ]:
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred).round(3))  
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred).round(3))  
print('Root Mean Squared:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)).round(3))
print('r2_score:', r2_score(y_test, y_pred).round(3))
print('Accuracy:', metrics.accuracy_score(y_test.astype('int'), y_pred).round(3))

# Hyperparameter Tuning on Random Forest

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
print(random_grid)

In [ ]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestClassifier()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random.fit(X, y)

In [ ]:
rf_random.best_params_

# Fitting the best parameter on Random Forest

In [ ]:
rf1 = RandomForestClassifier(n_estimators= 1200,
 min_samples_split= 8,
 min_samples_leaf= 4,
 max_features= 'auto',
 max_depth= None,
 bootstrap= False)
model_rf1 = rf1.fit(X_train, y_train)
y_pred_rf1 = model_rf1.predict(X_test)

from sklearn.metrics import accuracy_score
accu_score_rf1 = accuracy_score(y_test, y_pred_rf1)

print('accu_score_rf1:', accu_score_rf1)

# Converting Test data in Standard Scaler

In [ ]:
test = scaler.transform(test)

In [ ]:
y_pred_test = model_rf1.predict(test)

# Predicted Price Range on Test Data Set

In [ ]:
df_pred_test = pd.DataFrame({'Predicted': y_pred_test})
df_pred_test.head(10)


# Dont forget to like /vote